In [1]:
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

# Testes Modulares

### Geração de Massa

In [3]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'etapa':['FILTROS POLITICA'],
                             'flagErroAurora':[False],
                             'flagErroVisaoCliente':[False],
                             "marcacaoMesa":["XPTO"],
                             'solicitante.limites.limiteDisponivel':[10000],
                             'solicitante.limites.valorContratadoCartao':[0],
                             'solicitante.maxDiasAtraso':[0],
                             'solicitante.indFNV':[False],
                             'solicitante.indReneg':[False]
                             })

In [4]:
# Teste de casos de erro na chamada de origens
massa_validacaoErros = sherlock.Gera_Massa(base_entrada)
massa_validacaoErros.gera_cenario('etapa',['FILTROS POLITICA'])
massa_validacaoErros.gera_cenario('flagErroAurora',[False]) # a regra de erro bacen é executada no decision flow, testaremos no e2e
massa_validacaoErros.gera_cenario('flagErroVisaoCliente',[False]) # a regra de erro bacen é executada no decision flow, testaremos no e2e
massa_validacaoErros = massa_validacaoErros.get_massa_final()

# Teste de erro nos dados 
massa_validacaoeErroEDados = sherlock.Gera_Massa(base_entrada)
massa_validacaoeErroEDados.gera_cenario('etapa',['FILTROS POLITICA'])
massa_validacaoeErroEDados.gera_cenario('marcacaoMesa',['-99999',"XPTO","MESA"])
massa_validacaoeErroEDados.gera_cenario('solicitante.limites.limiteDisponivel',[-1,0,1,10000,-99999])
massa_validacaoeErroEDados.gera_cenario('solicitante.limites.valorContratadoCartao',[-1,0,1,-99999])
massa_validacaoeErroEDados.gera_cenario('solicitante.maxDiasAtraso',[0,4,5,6,-99999])
massa_validacaoeErroEDados.gera_cenario('solicitante.indFNV',[True,False])
massa_validacaoeErroEDados.gera_cenario('solicitante.indReneg',[True,False])
massa_validacaoeErroEDados = massa_validacaoeErroEDados.get_massa_final()

massa_validacaoInterna = sherlock.Gera_Massa(base_entrada)
massa_validacaoInterna.empilha_massas([massa_validacaoErros,
                                     massa_validacaoeErroEDados
                                     ])

massa_validacaoInterna.set_bom_arqv()
massa_validacaoInterna.converte_massa()

--------------------------------------
Quantidade de Linhas da Massa: 1202
--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
#Execução modular
listaResultado = rp.execucaoModular(massa_validacaoInterna.get_massa_json(),rp.validacoesInternas)

In [6]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['passouValidacaoInterna'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)

freqzinha = freqzinha.sort_values('valor', ascending=True)
freqzinha

,valor,Freq
0,0,414
9,1,48
8,2,80
1,3,320
2,4,160
5,7,12
7,8,12
6,10,12
4,11,48
3,12,96


In [7]:
# Renomeação das colunas para o cabeçalho do RMA
massa_validacaoInterna.rename_cabecalho()

#Contagem da qtd de regras negativas em que o cliente se enquadrou
tam = 0
for x in listaResultado:
    t = len(x['payloadHomol']['saidas']['regrasNegativas'])
    x['payloadHomol']['saidas']['qtdNegativas'] = t
    if t > tam:
        tam = t

print("tam: "+str(tam))

for y in range (0,tam):
    massa_validacaoInterna.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".nomeRegra","regrasNegativas#"+str(y+1)+".nomeRegra")
    massa_validacaoInterna.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".descricao","regrasNegativas#"+str(y+1)+".descRegra")
    
    massa_validacaoInterna.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".nome","listaLogs#"+str(y+1)+".nome")
    massa_validacaoInterna.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".valor","listaLogs#"+str(y+1)+".valor")

massa_validacaoInterna.atribui_expected(listaResultado,'payloadHomol.saidas.statusDecisao','statusDecisao')
massa_validacaoInterna.atribui_expected(listaResultado,'payloadHomol.saidas.vlrLimiteAprovado','vlrLimiteAprovado')
massa_validacaoInterna.atribui_expected(listaResultado,'payloadHomol.saidas.qtdNegativas','versionamentoPolitica')
# massa_validacaoInterna.atribui_expected(listaResultado,'payloadHomol.saidas.listaLogs','ListaLogs')

massa_validacaoInterna.get_massa_dm()

tam: 5


,Data-ID,Result-State,Result-Detail,payloadParam.etapa,flagErroAurora,flagErroVisaoCliente,marcacaoMesa,limiteDisponivel,valorContratadoCartao,maxDiasAtraso,...,result:listaLogs#5.nome,expected:listaLogs#5.nome,result:listaLogs#5.valor,expected:listaLogs#5.valor,result:statusDecisao,expected:statusDecisao,result:vlrLimiteAprovado,expected:vlrLimiteAprovado,result:versionamentoPolitica,expected:versionamentoPolitica
0,1,,,FILTROS POLITICA,false,false,XPTO,10000,0,0,...,,,,,,APROVADO,,0,,0
1,2,,,FILTROS POLITICA,false,false,XPTO,10000,0,0,...,,,,,,APROVADO,,0,,0
2,3,,,FILTROS POLITICA,false,false,-99999,-1,-1,0,...,,,,,,NEGADO,,0,,1
3,4,,,FILTROS POLITICA,false,false,-99999,-1,-1,0,...,,,,,,NEGADO,,0,,1
4,5,,,FILTROS POLITICA,false,false,-99999,-1,-1,0,...,,,,,,NEGADO,,0,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,1198,,,FILTROS POLITICA,false,false,MESA,-99999,-99999,6,...,,,,,,APROVADO,,0,,0
1198,1199,,,FILTROS POLITICA,false,false,MESA,-99999,-99999,-99999,...,,,,,,APROVADO,,0,,0
1199,1200,,,FILTROS POLITICA,false,false,MESA,-99999,-99999,-99999,...,,,,,,APROVADO,,0,,0
1200,1201,,,FILTROS POLITICA,false,false,MESA,-99999,-99999,-99999,...,,,,,,APROVADO,,0,,0


### Conversão da massa para CSV

In [8]:
massa_validacaoInterna.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaValidacoesInterna.csv',index=False)

#